## Imports

In [1]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config
from sklearn.impute import KNNImputer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [2]:
CLOUD = False

if CLOUD:
    import os
    os.environ['KAGGLE_USERNAME'] = "umutakta"
    os.environ['KAGGLE_KEY']      = "a471e5474add8d57cadac3cc5ff2b1d6"  # See https://www.kaggle.com/docs/api
    !pip install --upgrade kaggle
    !kaggle competitions download -c titanic
    DATA_PATH = "/work/Strive_School/Feature_Engineering_(Module_4)/Day_2_ Data Cleaning/titanic/"

else:
    DATA_PATH = "/work/Data_Sets/titanic/"

DATA_PATH

'/work/Data_Sets/titanic/'

## Load data

In [3]:
df      = pd.read_csv(DATA_PATH + "train.csv", index_col='PassengerId')
df_test = pd.read_csv(DATA_PATH + "test.csv",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

Train DataFrame: (891, 11)
Test DataFrame:  (418, 10)


## Check missings

In [4]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [5]:
df_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

# Exercise 1 (2pts):
Extract the title (Mr, Mrs, ... ) from the "Name" column.

Tips:
- split(',')[1] to get the 2nd part, and remove the surnamename
- split('.')[0] to get the 1str part, and remove the name

In [6]:
# CODE HERE get_Title_from_Name funtion
# Create this function using lambda (not def)

get_Title_from_Name = lambda x: x.split(",")[1].split(".")[0].strip()

# YOUR CODE HERE


df['Title']      = df['Name'].map(get_Title_from_Name )
df_test['Title'] = df_test['Name'].map(get_Title_from_Name)

In [7]:
assert df['Title'].values[0] == "Mr"
assert df['Title'].values[1] == "Mrs"
assert df['Title'].values[2] == "Miss"

assert df_test['Title'].values[0] == "Mr"
assert df_test['Title'].values[1] == "Mrs"
assert df_test['Title'].values[414] == "Dona"

# Exercise 2 (1pts):
Apply the title_dictionary to get a better information about the title. You have to overwrite the Title variable.

In [8]:
title_dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}

In [9]:
# Use map to apply the prevous dict

# df["Title"] =  
# df_test["Title"] =

# YOUR CODE HERE
df["Title"] = df['Title'].map(title_dictionary)
df_test["Title"] = df_test['Title'].map(title_dictionary)

In [10]:
assert df['Title'].values[886] == "Officer"
assert df_test['Title'].values[417] == "Master"

# Exercise OPTINAL (0pts):
Try to extract some information from the feature **Ticket**. Search on Internet if that colum has some kind of information.

In [11]:
df_test['Ticket']

PassengerId
892                 330911
893                 363272
894                 240276
895                 315154
896                3101298
               ...        
1305             A.5. 3236
1306              PC 17758
1307    SOTON/O.Q. 3101262
1308                359309
1309                  2668
Name: Ticket, Length: 418, dtype: object

# Exercise OPTIONAL (0pts):
Try to extract some information from the feature **Cabin**. Search on Internet if that colum has some kind of information.

In [12]:
df_test['Cabin']

df.Cabin.fillna(0, inplace=True) #We named Null cells as 'Empty'
df_test.Cabin.fillna(0, inplace=True) #We named Null cells as 'Empty'

df.loc[df["Cabin"] != 0, "Cabin"] = 1
df_test.loc[df_test["Cabin"] != 0, "Cabin" ] =1




In [13]:
df.Age.fillna(df.groupby(['Pclass', 'Sex']).Age.transform('mean'), inplace=True)
df_test.Age.fillna(df_test.groupby(['Pclass', 'Sex']).Age.transform('mean'), inplace=True)

# Preprocessing
For X data, notice that...
- We drop Survived because is the target variable
- We drop Name because we have extracted the Title: Mr, Mrs, ...
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [73]:
x = df.drop(columns=["Survived", 'Name', 'Ticket']) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test.drop(columns=['Name', 'Ticket']) # # X_TEST DATA (NEW DATA)

In [15]:
cat_vars  = ['Sex', 'Embarked', 'Title']         # x.select_dtypes(include=[object]).columns.values.tolist()
num_vars  = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age'] # x.select_dtypes(exclude=[object]).columns.values.tolist()

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']

Categorical features:
 ['Sex', 'Embarked', 'Title']


# Exercise 3 (2pts):
Create a **ColumnTransformer for Tree Models**. You need to create 2 pipelines (one for numerical and other for categories). Remember:
- Categorical pipeline: Some SimpleImputer -> Some Encoder
- Numerical pipeline: Some SimpleImputer -> NO Encoder

In [16]:

cat_4_multModels = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
    ('one hot', preprocessing.OneHotEncoder(
))
])


num_4_multmodels=pipeline.Pipeline(steps=[
    ("imputer",KNNImputer(n_neighbors=5)),
    ("quantile transformer",preprocessing.QuantileTransformer(n_quantiles=100, random_state=0))
    ])

tree_prepro = compose.ColumnTransformer(transformers=[
    #('num', num_4_treeModels, num_vars),
    #('cat', cat_4_treeModels, cat_vars),
    ("cat mult",cat_4_multModels,cat_vars),
    ("num mult",num_4_multmodels,num_vars)
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro



ColumnTransformer(transformers=[('cat mult',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('one hot', OneHotEncoder())]),
                                 ['Sex', 'Embarked', 'Title']),
                                ('num mult',
                                 Pipeline(steps=[('imputer', KNNImputer()),
                                                 ('quantile transformer',
                                                  QuantileTransformer(n_quantiles=100,
                                                                      random_state=0))]),
                                 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age'])])

# Exercise 4 (1pts):
1. Complete the diccionary with some Tree Models.
2. Then we put each model in a Pipeline where:
   - first is the prepocessing with the column Transformer
   - Then is the Tree model
3. Display the fullpipeline of the LGBMClassifier

In [18]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

In [19]:
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

In [20]:
mult_classifiers={
    "SVM": svm.SVC(),
    "Logistic": LogisticRegression(),
    "KNN":KNeighborsClassifier(n_neighbors=3),
    "MLP":MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)}

mult_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in mult_classifiers.items()}

mult_classifiers["SVM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat mult',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('one hot',
                                                                   OneHotEncoder())]),
                                                  ['Sex', 'Embarked', 'Title']),
                                                 ('num mult',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('quantile '
                                                                   'transformer',
                                                                   QuantileTransformer(n_quantiles=100,
                                                                                       random_state=0))]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age'])])),
                ('svc', SVC())])

In [21]:

x_train, x_val, y_train, y_val = model_selection.train_test_split(x,y, test_size=0.2,stratify = y, random_state=4)




results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

for model_name, model in mult_classifiers.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    model.fit(x_train,y_train)
    # GET PREDICTIONS USING x_val
    pred = model.predict(x_val)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              





results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


,Model,Accuracy,Bal Acc.,Time
1,MLP,84.357542,82.411067,0.242119
2,SVM,83.240223,80.421607,0.038903
3,Logistic,83.240223,81.501976,0.040529
4,KNN,81.005587,78.873518,0.042733


In [22]:

from sklearn.model_selection import cross_val_predict

skf = model_selection.StratifiedKFold(n_splits=10, random_state=0, shuffle=True)




results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})





for model_name, model in mult_classifiers.items():
    start_time = time.time()


        
    # TRAIN AND GET PREDICTIONS USING cross_val_predict() and x,y
    pred = cross_val_predict(model, x,y, cv=skf)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              


# YOUR CODE HERE



results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')


/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (

,Model,Accuracy,Bal Acc.,Time
1,SVM,81.705948,78.595053,0.475997
2,MLP,81.705948,78.980922,1.788326
3,Logistic,81.257015,79.333237,0.697373
4,KNN,80.134680,78.532739,0.509419


In [23]:

tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees": ExtraTreesClassifier(),
  "Random Forest": RandomForestClassifier(),
  "AdaBoost": AdaBoostClassifier(),
  "Skl GBM": GradientBoostingClassifier(),
  "Skl HistGBM": GradientBoostingClassifier(),
  "XGBoost": XGBClassifier(),
  "LightGBM": LGBMClassifier(),
  "CatBoost": CatBoostClassifier() }


# YOUR CODE HERE


tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}

tree_classifiers["LightGBM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat mult',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('one hot',
                                                                   OneHotEncoder())]),
                                                  ['Sex', 'Embarked', 'Title']),
                                                 ('num mult',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('quantile '
                                                                   'transformer',
                                                                   QuantileTransformer(n_quantiles=100,
                                                                                       random_state=0))]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age'])])),
                ('lgbmclassifier', LGBMClassifier())])

In [24]:
for pipe in tree_classifiers.values():
    assert type(pipe) is pipeline.Pipeline

# Exercise 5 (3pts):
Define a simple split validation strategy with:
- 80% for train
- 20% for validation
- With stratification
- random_state=0

And train all the models in a for loop

In [25]:

x_train, x_val, y_train, y_val = model_selection.train_test_split(x,y, test_size=0.2,stratify = y, random_state=4)




results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

for model_name, model in tree_classifiers.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    model.fit(x_train,y_train)
    # GET PREDICTIONS USING x_val
    pred = model.predict(x_val)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              





results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

1:	learn: 0.6820801	total: 49.2ms	remaining: 24.5s
2:	learn: 0.6770935	total: 50.2ms	remaining: 16.7s
3:	learn: 0.6723015	total: 51.3ms	remaining: 12.8s
4:	learn: 0.6672715	total: 52.3ms	remaining: 10.4s
5:	learn: 0.6625852	total: 53.4ms	remaining: 8.84s
6:	learn: 0.6576968	total: 54.4ms	remaining: 7.71s
7:	learn: 0.6527901	total: 55.4ms	remaining: 6.87s
8:	learn: 0.6472399	total: 56.5ms	remaining: 6.22s
9:	learn: 0.6421550	total: 57.5ms	remaining: 5.7s
10:	learn: 0.6369511	total: 58.6ms	remaining: 5.26s
11:	learn: 0.6320371	total: 59.6ms	remaining: 4.91s
12:	learn: 0.6282963	total: 60.7ms	remaining: 4.61s
13:	learn: 0.6246123	total: 61.3ms	remaining: 4.32s
14:	learn: 0.6201889	total: 62.3ms	remaining: 4.09s
15:	learn: 0.6164933	total: 63.1ms	remaining: 3.88s
16:	learn: 0.6118215	total: 64.1ms	remaining: 3.71s
17:	learn: 0.6078144	total: 65.2ms	remaining: 3.56s
18:	learn: 0.6040424	total: 66.2ms	remaining: 3.42s
19:	learn: 0.5998107	total: 67.3ms	remaining: 3.3s
20:	learn: 0.5954308	to

,Model,Accuracy,Bal Acc.,Time
1,CatBoost,87.150838,85.764163,1.398042
2,Skl GBM,85.474860,83.860343,0.120197
3,Skl HistGBM,85.474860,83.860343,0.131340
4,AdaBoost,83.240223,82.852437,0.111628
5,LightGBM,83.240223,82.312253,0.121211
6,XGBoost,82.681564,81.857708,0.149091
7,Random Forest,81.564246,79.868248,0.196945
8,Extra Trees,80.446927,78.418972,0.154289
9,Decision Tree,78.770950,77.325428,0.025628


# Exercise 6 (3pts):
Define a 10 Fold cross validation strategy with:
- With stratification
- shuffle=True
- random_state=0

And train all the models in a for loop.

Tip you can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)** for both training and predict with 

In [26]:

from sklearn.model_selection import cross_val_predict

skf = model_selection.StratifiedKFold(n_splits=10, random_state=0, shuffle=True)




results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})





for model_name, model in tree_classifiers.items():
    start_time = time.time()


        
    # TRAIN AND GET PREDICTIONS USING cross_val_predict() and x,y
    pred = cross_val_predict(model, x,y, cv=skf)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              


# YOUR CODE HERE



results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')


1:	learn: 0.6801017	total: 9.26ms	remaining: 4.62s
2:	learn: 0.6748498	total: 12ms	remaining: 4s
3:	learn: 0.6714246	total: 14ms	remaining: 3.49s
4:	learn: 0.6657903	total: 15.8ms	remaining: 3.14s
5:	learn: 0.6600028	total: 18.3ms	remaining: 3.03s
6:	learn: 0.6549985	total: 20ms	remaining: 2.84s
7:	learn: 0.6502942	total: 23.3ms	remaining: 2.89s
8:	learn: 0.6449366	total: 26.5ms	remaining: 2.92s
9:	learn: 0.6408220	total: 28.1ms	remaining: 2.78s
10:	learn: 0.6348829	total: 30.5ms	remaining: 2.74s
11:	learn: 0.6304520	total: 32.3ms	remaining: 2.66s
12:	learn: 0.6254857	total: 34ms	remaining: 2.58s
13:	learn: 0.6208682	total: 36.8ms	remaining: 2.59s
14:	learn: 0.6165273	total: 39.5ms	remaining: 2.59s
15:	learn: 0.6110367	total: 42.7ms	remaining: 2.63s
16:	learn: 0.6063400	total: 44.8ms	remaining: 2.59s
17:	learn: 0.6019598	total: 47.7ms	remaining: 2.6s
18:	learn: 0.5971015	total: 50.4ms	remaining: 2.6s
19:	learn: 0.5935575	total: 53ms	remaining: 2.6s
20:	learn: 0.5894821	total: 55.1ms	re

,Model,Accuracy,Bal Acc.,Time
1,Skl GBM,83.501684,81.485476,1.343295
2,Skl HistGBM,83.501684,81.485476,1.363973
3,CatBoost,82.940516,80.699358,16.969239
4,LightGBM,82.379349,80.629853,1.086854
5,XGBoost,81.930415,80.541175,1.606801
6,AdaBoost,81.705948,80.469274,1.172006
7,Random Forest,80.808081,79.134311,2.198112
8,Extra Trees,80.471380,78.916211,1.812095
9,Decision Tree,80.246914,78.734062,0.208418


**Training with all data the best model**

In [53]:
import optuna
import sklearn
from sklearn import datasets
def objective(trial):

    skf = model_selection.StratifiedKFold(n_splits=10, random_state=0, shuffle=True)

    tree_prepro.fit_transform(x)

    n_estimators = trial.suggest_int('n_estimators', 2, 20)

    max_depth = int(trial.suggest_loguniform('max_depth', 1, 32))
    clf=pipeline.make_pipeline(tree_prepro, GradientBoostingClassifier(n_estimators=n_estimators, max_depth=max_depth))


    return sklearn.model_selection.cross_val_score(clf, x, y, cv=skf).mean()


In [49]:
clf = DecisionTreeClassifier()
sklearn.model_selection.cross_val_score(clf, x, y, cv=5).mean()

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/tree/_classes.py", line 907, in fit
    X_idx_sorted=X_idx_sorted)
  File "/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/tree/_classes.py", line 159, in fit
    check_y_params))
  File "/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/base.py", line 430, in _validate_data
    X = check_array(X, **check_X_params)
  File "/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/utils/validation.py", line 63, in inner_f
    return f(*args, **kwargs)
 

nan

In [54]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2021-05-11 15:12:53,364] A new study created in memory with name: no-name-92059e0f-f1fa-43a3-b179-075cb3c09a18
[I 2021-05-11 15:12:54,251] Trial 0 finished with value: 0.7990886392009988 and parameters: {'n_estimators': 9, 'max_depth': 13.780976922527666}. Best is trial 0 with value: 0.7990886392009988.
[I 2021-05-11 15:12:54,614] Trial 1 finished with value: 0.8091510611735331 and parameters: {'n_estimators': 13, 'max_depth': 2.572290214598924}. Best is trial 1 with value: 0.8091510611735331.
[I 2021-05-11 15:12:54,942] Trial 2 finished with value: 0.7855680399500624 and parameters: {'n_estimators': 12, 'max_depth': 1.0099102854055846}. Best is trial 1 with value: 0.8091510611735331.
[I 2021-05-11 15:12:55,225] Trial 3 finished with value: 0.7866916354556804 and parameters: {'n_estimators': 10, 'max_depth': 1.265093292779183}. Best is trial 1 with value: 0.8091510611735331.
[I 2021-05-11 15:12:55,598] Trial 4 finished with value: 0.7855930087390763 and parameters: {'n_estimators': 

In [56]:
trial = study.best_trial
print('Accuracy: {}'.format(trial.value))
study.best_params

Accuracy: 0.8406242197253432


{'n_estimators': 7, 'max_depth': 4.862786659333275}

In [63]:
best_model = tree_classifiers["XGBoost"]

# Fit best model with all data

best_pipe=pipeline.make_pipeline(tree_prepro, GradientBoostingClassifier(n_estimators= 7, max_depth= 4.862786659333275))
best_pipe.fit(x,y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat mult',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('one hot',
                                                                   OneHotEncoder())]),
                                                  ['Sex', 'Embarked', 'Title']),
                                                 ('num mult',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('quantile '
                                                                   'transformer',
                                                                   QuantileTransformer(n_quantiles=100,
                                                                                       random_state=0))]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age'])])),
                ('gradientboostingclassifier',
                 GradientBoostingClassifier(max_depth=4.862786659333275,
                                            n_estimators=7))])

# Exercise 7.2 (2pts)
With your best model, generate the predicitions for test data (x_test)

In [79]:
x_test.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Cabin       0
Embarked    0
Title       0
dtype: int64

In [78]:
x_test.Title=x_test.Title.fillna(x_test.Title.mode()[0])
x_test.Fare=x_test.Fare.fillna(x_test.Fare.mean())

In [80]:
x_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
PassengerId,,,,,,,,,
892,3,male,34.500000,0,0,7.8292,0,Q,Mr
893,3,female,47.000000,1,0,7.0000,0,S,Mrs
894,2,male,62.000000,0,0,9.6875,0,Q,Mr
895,3,male,27.000000,0,0,8.6625,0,S,Mr
896,3,female,22.000000,1,1,12.2875,0,S,Mrs
...,...,...,...,...,...,...,...,...,...
1305,3,male,24.525104,0,0,8.0500,0,S,Mr
1306,1,female,39.000000,0,0,108.9000,1,C,Mr
1307,3,male,38.500000,0,0,7.2500,0,S,Mr


In [81]:
test_pred = best_pipe.predict(x_test)


# Exercise 7.3

Submit to kaggle.

- You can use the kaggle command line app. Check https://github.com/Kaggle/kaggle-api

In [82]:
sub = pd.DataFrame(test_pred, index=x_test.index, columns=["Survived"])
sub.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [83]:
sub.to_csv("sub.csv")

In [84]:
!kaggle competitions submit -c titanic -f sub.csv -m "My submission message"

Traceback (most recent call last):
  File "/root/venv/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/root/venv/lib/python3.7/site-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/root/venv/lib/python3.7/site-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=796ba814-f632-4502-b0d7-7e2f8f9e546d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>